In [1]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import json, os, pickle, glob#, natsort
import corner
import astropy.units as u

In [3]:
import la_forge.diagnostics as dg
import la_forge.core as co
from la_forge.utils import bayes_fac
from targeted_cws_ng15 import post_processing as pp

In [4]:
project_path = '/vast/palmer/home.grace/bbl29/targeted_cws_ng15'
outdir_path = '/vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains'
dataset = 'ng15_v1p1'
c_light = 299792458 # m/s
G = 6.67430e-11 # Nm^2/kg

In [6]:
target_paths = glob.glob(f'{outdir_path}/{dataset}/*_UL') + glob.glob(f'{outdir_path}/{dataset}/*_UL_varyfgw')
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if 'altskyloc' in tp]
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if 'altskyloc' in tp]
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if 'nHz' in tp]
targets = [tp.replace(f'{outdir_path}/{dataset}/','').replace('_UL','') for tp in target_paths]
targets_copy = np.copy(targets)
targets_og = {}
Nt = len(targets)
for i in range(Nt):
    targets[i] = targets[i].replace('_',' ')
    targets[i] = targets[i].replace('-','$-$')
    if 'varyfgw' in targets[i]:
        targets[i] = targets[i].replace(' varyfgw','')
    targets_og[targets[i]] = targets_copy[i]
    #if '16nHz' in targets[i]:
    #    targets[i] = targets[i].replace(' 16nHz','')
    #if '49nHz' in targets[i]:
    #    targets[i] = targets[i].replace(' 49nHz','')
targets = sorted(targets)
target_paths = sorted(target_paths)
print(Nt,'targets')
print(targets)

16 targets
['3C66B', 'HS 0926+3608', 'HS 1630+2355', 'NGC 3115', 'OJ287', 'PKS 2131$-$021', 'PKS J0805$-$0111', 'SDSS J092911.35+2037', 'SDSS J114857.33+1600', 'SDSS J131706.19+2714', 'SDSS J133516.17+1833', 'SDSS J134855.27$-$0321', 'SDSS J140704.43+2735', 'SDSS J160730.33+1449', 'SDSS J164452.71+4307', 'SNU J13120+0641']


In [7]:
target_paths_det = glob.glob(f'{outdir_path}/{dataset}/*_det*')
[target_paths_det.pop(target_paths_det.index(tp)) for tp in np.flip(target_paths_det) if 'altskyloc' in tp]
[target_paths_det.pop(target_paths_det.index(tp)) for tp in np.flip(target_paths_det) if 'altskyloc' in tp]
[target_paths_det.pop(target_paths_det.index(tp)) for tp in np.flip(target_paths_det) if '49nHz' in tp]
target_paths_det = sorted(target_paths_det)

Load cores

In [8]:
c_UL = {}
for t, tp in zip(targets, target_paths):
    print(t, tp)
    #if not t == 'SDSS J131706.19+2714':
    #    continue
    c_UL[t] = co.Core(corepath=f'{tp}/core.h5', label=t)

3C66B /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/3C66B_UL
HS 0926+3608 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/HS_0926+3608_UL
HS 1630+2355 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/HS_1630+2355_UL
NGC 3115 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/NGC_3115_UL_varyfgw
OJ287 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/OJ287_UL
PKS 2131$-$021 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/PKS_2131-021_UL
PKS J0805$-$0111 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/PKS_J0805-0111_UL
SDSS J092911.35+2037 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/SDSS_J092911.35+2037_UL
SDSS J114857.33+1600 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/SDSS_J114857.33+1600_UL
SDSS J131706.19+2714 /vas

In [9]:
c_det = {}
for t, tp in zip(targets[len(c_det.keys()):], target_paths_det[len(c_det.keys()):]):
    print(t, tp)
    c_det[t] = co.Core(corepath=f'{tp}/core.h5', label=t)

3C66B /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/3C66B_det
HS 0926+3608 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/HS_0926+3608_det
HS 1630+2355 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/HS_1630+2355_det
NGC 3115 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/NGC_3115_det_varyfgw
OJ287 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/OJ287_det
PKS 2131$-$021 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/PKS_2131-021_det
PKS J0805$-$0111 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/PKS_J0805-0111_det
SDSS J092911.35+2037 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/SDSS_J092911.35+2037_det
SDSS J114857.33+1600 /vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains/ng15_v1p1/SDSS_J114857.33+1600_det
SDSS J131706.19+

Get Bayes factors and upper limits. Note there is a bit of difference in the Bayes factors depending whether you use $\mathcal{M}_c$ or $h_0$ to calculate this, so we will ultimately report everything just to 1 decimal point.

In [10]:
BF = {}
log10_h0_UL = {}
log10_mc_UL = {}
for t in targets:
    BF[t] = pp.get_bayes_fac(c_det[t], amp_param='log10_mc')
    #if t == 'SDSS J131706.19+2714':
    #    log10_mc_UL[t] = c_det[t].get_param_credint('log10_mc', onesided=True, interval=95)
    #    log10_h0_UL[t] = c_det[t].get_param_credint('log10_h0', onesided=True, interval=95)
    #    continue
    log10_mc_UL[t] = c_UL[t].get_param_credint('log10_mc', onesided=True, interval=95)
    log10_h0_UL[t] = c_UL[t].get_param_credint('log10_h0', onesided=True, interval=95)

Make table

In [11]:
table = []
for i, t in enumerate(targets):
    line = []
    line.append(t)
    line.append('')
    line.append('')
    line.append('')
    line.append(f'{np.round(log10_mc_UL[t], decimals=2)}')
    line.append(f'{np.round(log10_h0_UL[t], decimals=2)}')
    line.append(f'{np.round(BF[t], decimals=1)}')
    fullline = ' & '.join(line)
    fullline += r' \\'
    table.append(fullline)
print('\n'.join(table))

3C66B &  &  &  & 8.98 & -13.88 & 0.3 \\
HS 0926+3608 &  &  &  & 10.55 & -13.93 & 1.3 \\
HS 1630+2355 &  &  &  & 10.15 & -14.16 & 0.8 \\
NGC 3115 &  &  &  & 8.64 & -13.39 & 1.4 \\
OJ287 &  &  &  & 9.98 & -14.12 & 0.6 \\
PKS 2131$-$021 &  &  &  & 10.2 & -14.28 & 0.8 \\
PKS J0805$-$0111 &  &  &  & 10.37 & -13.93 & 1.0 \\
SDSS J092911.35+2037 &  &  &  & 10.53 & -13.93 & 0.9 \\
SDSS J114857.33+1600 &  &  &  & 10.26 & -14.17 & 0.7 \\
SDSS J131706.19+2714 &  &  &  & 10.38 & -14.34 & 0.8 \\
SDSS J133516.17+1833 &  &  &  & 10.12 & -14.36 & 0.6 \\
SDSS J134855.27$-$0321 &  &  &  & 10.47 & -14.02 & 0.8 \\
SDSS J140704.43+2735 &  &  &  & 10.41 & -14.19 & 1.1 \\
SDSS J160730.33+1449 &  &  &  & 10.23 & -14.4 & 0.7 \\
SDSS J164452.71+4307 &  &  &  & 10.37 & -14.19 & 0.8 \\
SNU J13120+0641 &  &  &  & 9.81 & -14.0 & 0.8 \\
